In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data_url = "https://raw.githubusercontent.com/fenago/tf/main/Chapter5-Classification_Models/datasets/accelerometer.csv"

data = pd.read_csv(data_url, sep=",")
data.head()

,wconfid,pctid,x,y,z
0,1,20,1.004,0.090,-0.125
1,1,20,1.004,-0.043,-0.125
2,1,20,0.969,0.090,-0.121
3,1,20,0.973,-0.012,-0.137
4,1,20,1.000,-0.016,-0.121


In [3]:
data['wconfid'].value_counts()

1    51000
2    51000
3    51000
Name: wconfid, dtype: int64

In [4]:
reduce_map = {"wconfid": {1:0, 2:1, 3:2} }
data.replace(reduce_map, inplace=True)

In [5]:
data['wconfid'].value_counts()

0    51000
1    51000
2    51000
Name: wconfid, dtype: int64

In [6]:
data.dropna(inplace=True)

In [7]:
y = data.pop('wconfid')
X = data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, 
                                                  stratify=y,random_state=42)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(107100, 4)
(45900, 4)
(107100,)
(45900,)


In [10]:
y_train.head()

74339     1
144853    2
48273     0
75741     1
122914    2
Name: wconfid, dtype: int64

In [11]:
import tensorflow as tf

In [12]:
tf.random.set_seed(8)

In [13]:
model = tf.keras.Sequential()

In [14]:
from tensorflow.keras.layers import Dense

In [15]:
fc1 = Dense(512, input_shape=(4,), activation='relu')

In [16]:
fc2 = Dense(512, activation='relu')

In [17]:
fc3 = Dense(128, activation='relu')

In [18]:
fc4 = Dense(128, activation='relu')

In [19]:
fc5 = Dense(3, activation='softmax')

In [20]:
model.add(fc1)
model.add(fc2)
model.add(fc3)
model.add(fc4)
model.add(fc5)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               2560      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 387       
Total params: 347,779
Trainable params: 347,779
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [23]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [24]:
model.compile(optimizer=optimizer, loss=loss, \
                  metrics=['accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
3347/3347 [==============================] - 7s 2ms/step - loss: 0.8896 - accuracy: 0.5044
Epoch 2/5
3347/3347 [==============================] - 6s 2ms/step - loss: 0.7556 - accuracy: 0.5953
Epoch 3/5
3347/3347 [==============================] - 6s 2ms/step - loss: 0.7204 - accuracy: 0.6210
Epoch 4/5
3347/3347 [==============================] - 6s 2ms/step - loss: 0.7028 - accuracy: 0.6326
Epoch 5/5
3347/3347 [==============================] - 6s 2ms/step - loss: 0.6916 - accuracy: 0.6385


In [26]:
model.evaluate(X_test, y_test)

1435/1435 [==============================] - 2s 1ms/step - loss: 0.7036 - accuracy: 0.6323


[0.7035843729972839, 0.6323093771934509]

In [27]:
from tensorflow.math import confusion_matrix

In [28]:
preds_proba = model.predict(X_test)

In [29]:
preds = preds_proba.argmax(axis=1)

In [30]:
confusion_matrix(y_test, preds)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 5355,  7150,  2795],
       [ 2431, 10400,  2469],
       [  168,  1864, 13268]])>